## Welcome!

Firstly, congratulations on advancing this far in the hiring process! The fact that you are reading this means that E&N has recognized your potential and is excited to test your skills further. We hope you find this task both challenging and enjoyable. Please feel free to share any feedback during your upcoming skills interview.

### Task Overview

This task is designed to test your problem-solving abilities and your familiarity with computational notebooks. While it is not intended to be easy, we are not expecting perfection. We value clear, concise documentation and a genuine attempt to produce functional code.

**Background:** In the future, when a data consumer (aka developer/customer) comes to The Graph and pays for data to be indexed,  a gateway will need to allocate a group of indexers (service providers) to index their subgraph. What follows is written from the point of view of a gateway operator. Indexers are given a score that shows how suitable we think they are to serve data on each chain. After a score has been assigned to all indexers on each chain we use indexers’ scores to inform selection probability of receiving an indexing agreement for each chain. An indexing agreement is an agreement that means the indexer will provide indexing services and we will pay the indexer for their indexing services (the customer pays us and we act as an intermediary). An indexer's aim is to receive as many indexing agreements as they can in order to maximise the income they generate from their indexing activities. Indexers that provide higher quality of service (QoS) at lower prices will increase their chances of receiving indexing agreements.

A single indexing agreement contains information such as the price the indexer will get paid, and what data they have to make available to be queried. We can award contracts that specify the portion of data which has to be served to several indexers, instead of a single indexer. By giving a group of indexers an indexing agreement instead of allocating that agreement to only 1 indexer we are aiming to provide customers (developers/data consumers) with a higher quality of service. The rationale is that if a single indexer inside the group was to have issues then other indexers in the group can continue to operate. Awarding indexing agreements to a group should help to improve quality of service, reduce query latency, increase data uptime and increase query success rate, among other things that a consumer may find valuable. However the QoS that a customer receives is a function of the indexers that are included in the group the customer is assigned. Your task is to figure out how to group indexers together for the best QoS for customers.

In this task we have provided you with a CSV containing an assortment of (simulated) indexers, where those indexers are located, what VPS provider the indexer is using, and the indexer's current score. We have also already computed how many indexing agreements each indexer should receive based on their score, for the next batch of 10,000 indexing agreements. To test your python skills we want you to group indexers into groups of 3 indexers per group.

The groups you create (in this hypothetical example) will be paid to index data for future customers, with each group having a chance to be awarded indexing agreements. When forming these groups, you should consider decentralisation and the overall quality of service (QoS) each group can provide. Another crucial factor is that the number of agreements allocated to the groups in which an indexer appears should sum up to the total number of agreements that indexer has been awarded for the batch. This ensures that, as groups are randomly selected to serve customers' data, the total number of agreements each indexer fulfils equilibrates to their targeted number over time. If you do decide to deviate from this requirement make sure to document your logic, as we are keen to understand your thought process.

You should also think about what you are optimising for when creating the groups and efficient ways to complete this task without spending too much time on it, although if you want to focus on the quality of your submission you are free to spend more time wherever you think is appropriate. Your submission will be compared against other candidates so you should aim to do your best work, but don't stress about it, this is far from the only selection criteria for this role. Let us know if you have any problems with this assignment in your upcoming skills interview. We are looking forward to meeting you!


### Key Points to Focus On

1. **Optimization:** Aim to optimise the indexer groups you create. Randomly allocating indexers into groups of 3 will not score many points.
2. **Documentation:** Ensure your code and your logic is well-documented where appropriate, making it easy to understand and follow.
3. **Plots**: Attempt to demonstrate your work visually with graphs where appropriate. How you chose to do this is left for you to decide. You are not required to use python.
4. **Optimization:** If possible, try to optimise the speed of your code execution. More care should be taken to perform sound analysis, than writing efficient code, however writing efficient code is also beneficial.

You will notice that a start to the assignment has already been made, you are free to continue by adding to the existing code or start fresh with your own code. If you are having any trouble, or would like further clarity on anything, you can send as many emails as you like to samuel@edgeandnode.com, pablo@edgeandnode.com & rem@edgeandnode.com.

We understand that candidates might have different time availability for take home challenges - we hope you can dedicate between 2 and 4 hours to solving this, and that you can present your solution in a week or two; if this will not work for you please let us know and we'll do our best to accommodate.

We look forward to seeing your approach and solutions. Good luck!

Please direct your submission to [samuel\@edgeandnode.com](mailto:samuel@edgeandnode.com) (@MoonBoi9001 on GitHub), [pablo\@edgeandnode.com](mailto:pablo@edgeandnode.com) (@pcarranzav) & [rem\@edgeandnode.com](mailto:rem@edgeandnode.com)  (@RembrandK). You can create a private fork of this repository on GitHub, and add us as collaborators, or send us a compressed file with your solution.

#### Import relevant packages

In [1]:
# Imports
import pandas as pd
import numpy as np
from itertools import combinations

#### Load the example data you have been given for this asignment

In [12]:
indexer_df = pd.read_csv("Example_indexer_df_skills_take_home.csv")

# Display the first 4 and last 2 rows of the df
pd.concat([
    indexer_df.head(4),
    pd.DataFrame(['...'] * len(indexer_df.columns)).transpose().set_axis(indexer_df.columns, axis=1),
    indexer_df.tail(2)
])

,indexer,indexer_vps_provider,indexer_location,indexing_agreements,score
0,0x8fa75d001fab9f1c68e89afdd891fdaa61dfd704,AS24940 Hetzner Online GmbH,"60,20",262,0.910806
1,0x2b1ca69d13051e51da75ea0a9af5b4a6efe2418c,AS24940 Hetzner Online GmbH,"60,20",242,0.881894
2,0xd0d1e3045748f13c71ccd1b436abffaa608828b0,"AS13335 Cloudflare, Inc.","40,-120",233,0.871032
3,0xb85ef17d0bb06cf04aea8612725416c326dda7e5,AS16276 OVH SAS,"40,-120",210,0.818766
0,...,...,...,...,...
72,0x24e5e10ddb878e31cf88cac59d4576f0f4be9dd7,"AS13335 Cloudflare, Inc.","40,-120",6,0.503504
73,0xf67a0cbe27dda32dce76ae0dbe36c7cbcdb84a12,AS24940 Hetzner Online GmbH,"60,20",25,0.438394


#### 1. Create a new df containing permutations of indexers from the indexer_df, give each permutation a group score.
#### 2. Add any further columns to this df that might be useful to determine how suitable the group is.

In [13]:
# Create dictionary's for quick lookups
provider_dict = indexer_df.set_index('indexer')['indexer_vps_provider'].to_dict()
loc_dict = indexer_df.set_index('indexer')['indexer_location'].to_dict()
score_dict = indexer_df.set_index('indexer')['score'].to_dict()

# Calculate Possible Groupings and Their Scores.
groups = list(combinations(indexer_df['indexer'], 3))
group_scores = [score_dict[x] + score_dict[y] + score_dict[z] for x, y, z in groups]

In [14]:
group_df = pd.DataFrame({
    "group": groups,
    "score": group_scores
})

# Filter out groups with too low scores. We already know that we wont be using them. 
# In this case any group with lower group score than the 1% percentile group score is removed.

threshold_score = np.percentile(group_df['score'], 1)
group_df = group_df[group_df['score'] > threshold_score].reset_index(drop=True)

In [15]:
group_df

,group,score
0,"(0x8fa75d001fab9f1c68e89afdd891fdaa61dfd704, 0...",2.663732
1,"(0x8fa75d001fab9f1c68e89afdd891fdaa61dfd704, 0...",2.611467
2,"(0x8fa75d001fab9f1c68e89afdd891fdaa61dfd704, 0...",2.601804
3,"(0x8fa75d001fab9f1c68e89afdd891fdaa61dfd704, 0...",2.591989
4,"(0x8fa75d001fab9f1c68e89afdd891fdaa61dfd704, 0...",2.591233
...,...,...
64170,"(0x97ffbe611708a48637121a7a734d7345ecd3395b, 0...",1.654575
64171,"(0x97ffbe611708a48637121a7a734d7345ecd3395b, 0...",1.654340
64172,"(0x97ffbe611708a48637121a7a734d7345ecd3395b, 0...",1.653663
64173,"(0x97ffbe611708a48637121a7a734d7345ecd3395b, 0...",1.653631


#### Using your df above that contains permutations of indexers, their scores and their suitability, figure out which groups should receive the indexing agreements and how many agreements they should receive.

#### You may want to formulate a linear programming problem, however you are free to tackle this however you see fit. 